In [1]:
import sys
if "../src/" not in sys.path:
    sys.path.insert(0,"../src/")

In [2]:
import numpy as np
from pbcpy.formats.qepp import PP
from pbcpy.optimization import Optimization
from pbcpy.functionals import FunctionalClass, TotalEnergyAndPotential

Use numpy for Fourier Transform


### load guess density

In [3]:
mol = PP(filepp='Al_fde_rho.pp').read()

### Load KEDF

In [4]:
optional_kwargs = {}
optional_kwargs["Sigma"] = 0.0
optional_kwargs["x"] = 1.0
optional_kwargs["y"] = 1.0
KE = FunctionalClass(type='KEDF',name='x_TF_y_vW',is_nonlocal=False,optional_kwargs=optional_kwargs)

### load XC

In [5]:
XC = FunctionalClass(type='XC',name='LDA',is_nonlocal=False)

### load IONS and HARTREE

In [17]:
optional_kwargs = {}
optional_kwargs["PP_list"] = {'Al':'./Al_lda.oe01.recpot'}
optional_kwargs["ions"]    = mol.ions 

IONS = FunctionalClass(type='IONS', optional_kwargs=optional_kwargs)
HARTREE = FunctionalClass(type='HARTREE')

### load energy evaluator

In [18]:
E_v_Evaluator = TotalEnergyAndPotential(rho=mol.field,
                                        KineticEnergyFunctional=KE,
                                        XCFunctional=XC,
                                        HARTREE=HARTREE,
                                        IONS=IONS)

### instance optimizer

In [19]:
opt = Optimization(EnergyEvaluator=E_v_Evaluator, optimization_method = 'TN')

### optimize!

In [20]:
new_rho = opt.optimize_rho(guess_rho=mol.field)

func time 0.06056833267211914
Step    Energy(a.u.)            dE              dP              Nd      Nls     Time(s)         
0       1.427370978142E+00      +999999999E99   1.711157E+04    1       1       6.275773E-02    
1       2.184425105154E-02      -1.405527E+00   2.142150E+03    49      1       1.565764E+00    
2       -2.581575392204E-02     -4.766000E-02   1.312786E+02    12      1       2.019736E+00    
3       -4.751849753123E-02     -2.170274E-02   1.453647E+01    41      4       3.284698E+00    
4       -4.842189465565E-02     -9.033971E-04   1.295853E+00    17      1       3.882707E+00    
5       -4.855868840000E-02     -1.367937E-04   1.171214E-01    34      1       4.915240E+00    
6       -4.856528085973E-02     -6.592460E-06   1.082296E-02    23      1       5.606279E+00    
#### Density Optimization Converged ####


In [21]:
delta_rho = np.abs(new_rho - mol.field).integral()/2

In [22]:
print(delta_rho)

0.45533255345808193


In [23]:
E_v_Evaluator.Energy(rho=mol.field,ions=mol.ions)

-3.9344391154081597